# ArcLLM Step 1: Core Types & Exceptions

This notebook walks through everything built in Step 1 — the **type layer** that forms the contract every other layer builds on.

**What was built:**
- Exception hierarchy (`ArcLLMError`, `ArcLLMParseError`, `ArcLLMConfigError`)
- Content blocks (`TextBlock`, `ImageBlock`, `ToolUseBlock`, `ToolResultBlock`)
- Discriminated union (`ContentBlock`)
- `Message`, `Tool`, `ToolCall`, `Usage`, `LLMResponse`
- `LLMProvider` abstract base class

**Why it matters:** These types are the single source of truth. Every adapter, module, and agent interacts through them. Get these wrong, and everything downstream breaks.

In [ ]:
# First, make sure arcllm is importable
from arcllm import (
    TextBlock, ImageBlock, ToolUseBlock, ToolResultBlock,
    Message, Tool, ToolCall, Usage, LLMResponse, LLMProvider,
    ArcLLMError, ArcLLMParseError, ArcLLMConfigError,
    load_model,
)
print("All imports successful!")

---
## 1. Exception Hierarchy

ArcLLM has a clean 3-class exception tree:

```
ArcLLMError (base — catch everything)
├── ArcLLMParseError   (tool call JSON couldn't be parsed)
└── ArcLLMConfigError  (config validation failed)
```

**Design decision:** Fail fast, fail loud. Every error attaches raw data so agents can log/debug.

In [ ]:
# ArcLLMError is the base — all ArcLLM exceptions inherit from it
err = ArcLLMError("something went wrong")
print(f"Type: {type(err).__name__}")
print(f"Message: {err}")
print(f"Is Exception? {isinstance(err, Exception)}")

In [ ]:
# ArcLLMParseError — when an LLM returns unparseable tool call arguments
# It preserves the raw string AND the original error for debugging
import json

bad_json = '{"broken json'
try:
    json.loads(bad_json)
except json.JSONDecodeError as e:
    parse_err = ArcLLMParseError(raw_string=bad_json, original_error=e)

print(f"Error message: {parse_err}")
print(f"Raw string preserved: {parse_err.raw_string!r}")
print(f"Original error type: {type(parse_err.original_error).__name__}")
print(f"Is ArcLLMError? {isinstance(parse_err, ArcLLMError)}")

In [ ]:
# ArcLLMConfigError — raised when config validation fails
config_err = ArcLLMConfigError("Missing API key env var")
print(f"Message: {config_err}")
print(f"Is ArcLLMError? {isinstance(config_err, ArcLLMError)}")

# You can catch ALL arcllm errors with one except clause:
try:
    raise ArcLLMParseError("bad", ValueError("oops"))
except ArcLLMError as e:
    print(f"\nCaught via base class: {type(e).__name__}: {e}")

---
## 2. Content Blocks

LLM messages aren't just text. They can contain images, tool calls, and tool results.

ArcLLM models this with **4 content block types**, each tagged with a `type` field:

| Block | `type` | Purpose |
|-------|--------|---------|
| `TextBlock` | `"text"` | Plain text content |
| `ImageBlock` | `"image"` | Image data (base64 or URL) |
| `ToolUseBlock` | `"tool_use"` | LLM wants to call a tool |
| `ToolResultBlock` | `"tool_result"` | Result from executing a tool |

In [ ]:
# TextBlock — the simplest content type
text = TextBlock(text="Hello from Claude!")
print(f"Type tag: {text.type!r}")
print(f"Text: {text.text!r}")
print(f"As dict: {text.model_dump()}")

In [ ]:
# ImageBlock — for vision-capable models
image = ImageBlock(source="base64encodeddata...", media_type="image/png")
print(f"Type tag: {image.type!r}")
print(f"Media type: {image.media_type}")
print(f"As dict: {image.model_dump()}")

In [ ]:
# ToolUseBlock — when the LLM wants to call a tool
# This is the core of agentic behavior!
tool_use = ToolUseBlock(
    id="toolu_01abc",
    name="search_database",
    arguments={"query": "arcllm config", "limit": 10}
)
print(f"Type tag: {tool_use.type!r}")
print(f"Tool name: {tool_use.name}")
print(f"Arguments: {tool_use.arguments}")
print(f"Call ID: {tool_use.id}  (links to ToolResultBlock)")

In [ ]:
# ToolResultBlock — the agent's response after executing the tool
# Simple case: string content
result_simple = ToolResultBlock(
    tool_use_id="toolu_01abc",  # must match the ToolUseBlock.id
    content="Found 3 matching records"
)
print(f"Type tag: {result_simple.type!r}")
print(f"Links to: {result_simple.tool_use_id}")
print(f"Content: {result_simple.content!r}")

In [ ]:
# ToolResultBlock can also contain nested ContentBlocks!
# This is for rich results (text + images, multiple sections, etc.)
result_rich = ToolResultBlock(
    tool_use_id="toolu_01abc",
    content=[
        TextBlock(text="Found 3 records:"),
        TextBlock(text="1. arcllm.config — TOML loader"),
        TextBlock(text="2. arcllm.types — Core types"),
    ]
)
print(f"Content is a list: {type(result_rich.content).__name__}")
print(f"Number of blocks: {len(result_rich.content)}")
for i, block in enumerate(result_rich.content):
    print(f"  [{i}] {type(block).__name__}: {block.text}")

### The Discriminated Union

`ContentBlock` is a **discriminated union** — Pydantic looks at the `type` field to decide which model to use.

This means you can pass raw dicts with a `type` key and Pydantic auto-parses them into the right class.

In [ ]:
from pydantic import TypeAdapter
from arcllm.types import ContentBlock

# Parse raw dicts — Pydantic checks the 'type' field automatically
adapter = TypeAdapter(ContentBlock)

parsed_text = adapter.validate_python({"type": "text", "text": "hello"})
print(f"Dict -> {type(parsed_text).__name__}: {parsed_text.text}")

parsed_tool = adapter.validate_python({
    "type": "tool_use",
    "id": "c1",
    "name": "search",
    "arguments": {}
})
print(f"Dict -> {type(parsed_tool).__name__}: {parsed_tool.name}")

# Invalid type field will raise a ValidationError
from pydantic import ValidationError
try:
    adapter.validate_python({"type": "audio", "data": "..."})
except ValidationError as e:
    print(f"\nInvalid type rejected: {e.error_count()} error(s)")

---
## 3. Message

A `Message` combines a **role** and **content**. The role is one of four literals:

| Role | Who's talking | Typical content |
|------|---------------|-----------------|
| `"system"` | System prompt | Text instructions |
| `"user"` | The human or agent | Text, images |
| `"assistant"` | The LLM | Text, tool use blocks |
| `"tool"` | Tool execution | Tool result blocks |

Content can be a plain `str` OR a `list[ContentBlock]`.

In [ ]:
# Simple text message
user_msg = Message(role="user", content="What's the weather in Austin?")
print(f"Role: {user_msg.role}")
print(f"Content type: {type(user_msg.content).__name__}")
print(f"Content: {user_msg.content}")

In [ ]:
# Multi-block assistant message (text + tool call)
# This is what a real agentic response looks like
assistant_msg = Message(
    role="assistant",
    content=[
        TextBlock(text="Let me check the weather for you."),
        ToolUseBlock(
            id="toolu_weather_1",
            name="get_weather",
            arguments={"city": "Austin", "units": "fahrenheit"}
        ),
    ],
)
print(f"Role: {assistant_msg.role}")
print(f"Content blocks: {len(assistant_msg.content)}")
for block in assistant_msg.content:
    print(f"  {type(block).__name__}: ", end="")
    if isinstance(block, TextBlock):
        print(block.text)
    elif isinstance(block, ToolUseBlock):
        print(f"{block.name}({block.arguments})")

In [ ]:
# Discriminated union works inside Message too — raw dicts auto-parse
msg_from_dicts = Message(
    role="assistant",
    content=[
        {"type": "text", "text": "thinking..."},
        {"type": "tool_use", "id": "c1", "name": "search", "arguments": {}},
    ],
)
print(f"Parsed from dicts:")
print(f"  [0] {type(msg_from_dicts.content[0]).__name__}")
print(f"  [1] {type(msg_from_dicts.content[1]).__name__}")

In [ ]:
# Invalid roles are rejected at construction time — fail fast!
from pydantic import ValidationError

try:
    Message(role="admin", content="hack the planet")
except ValidationError as e:
    print(f"Rejected! {e.error_count()} validation error(s):")
    for err in e.errors():
        print(f"  Field: {err['loc']}, Type: {err['type']}")

In [ ]:
# Edge case: empty content list is allowed (some APIs use this)
empty_msg = Message(role="user", content=[])
print(f"Empty content list: {empty_msg.content}")

---
## 4. Tool

A `Tool` is the **definition** you send TO the LLM — "here are tools you can use."

The `parameters` field is raw JSON Schema (`dict[str, Any]`), kept loose intentionally so it matches what providers expect.

In [ ]:
# Define a tool the LLM can call
search_tool = Tool(
    name="search_database",
    description="Search the internal knowledge base for relevant documents",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query"
            },
            "limit": {
                "type": "integer",
                "description": "Max results to return",
                "default": 10
            }
        },
        "required": ["query"]
    }
)

print(f"Tool name: {search_tool.name}")
print(f"Description: {search_tool.description}")
print(f"Parameters schema: {json.dumps(search_tool.parameters, indent=2)}")

---
## 5. ToolCall

A `ToolCall` is what the LLM returns — "I want to call this tool with these arguments."

Key difference from `ToolUseBlock`: ToolCall is a standalone object on `LLMResponse.tool_calls`. ToolUseBlock is a content block inside a message.

In [ ]:
# ToolCall — what comes back from the LLM
call = ToolCall(
    id="toolu_01XYZ",
    name="search_database",
    arguments={"query": "federal compliance", "limit": 5}
)
print(f"Call ID: {call.id}")
print(f"Tool: {call.name}")
print(f"Arguments (already parsed dict): {call.arguments}")
print(f"\nNote: arguments is always a dict, never a raw JSON string.")
print(f"The adapter handles parsing. If it fails -> ArcLLMParseError.")

---
## 6. Usage

Every LLM response includes token usage. ArcLLM tracks:
- Required: `input_tokens`, `output_tokens`, `total_tokens`
- Optional: `cache_read_tokens`, `cache_write_tokens`, `reasoning_tokens`

Optional fields are `None` when the provider doesn't report them.

In [ ]:
# Basic usage (what most providers return)
basic_usage = Usage(input_tokens=1500, output_tokens=300, total_tokens=1800)
print(f"Input:  {basic_usage.input_tokens:,} tokens")
print(f"Output: {basic_usage.output_tokens:,} tokens")
print(f"Total:  {basic_usage.total_tokens:,} tokens")
print(f"Cache read:  {basic_usage.cache_read_tokens}  (None = not reported)")
print(f"Cache write: {basic_usage.cache_write_tokens}  (None = not reported)")
print(f"Reasoning:   {basic_usage.reasoning_tokens}  (None = not reported)")

In [ ]:
# Full usage with caching and reasoning (Anthropic-style)
full_usage = Usage(
    input_tokens=1500,
    output_tokens=300,
    total_tokens=1800,
    cache_read_tokens=1200,   # tokens read from prompt cache
    cache_write_tokens=300,   # tokens written to prompt cache
    reasoning_tokens=150,     # tokens used for extended thinking
)
print(f"Cache read:  {full_usage.cache_read_tokens:,} tokens")
print(f"Cache write: {full_usage.cache_write_tokens:,} tokens")
print(f"Reasoning:   {full_usage.reasoning_tokens:,} tokens")

In [ ]:
# Zero tokens is valid (edge case, but should not crash)
zero_usage = Usage(input_tokens=0, output_tokens=0, total_tokens=0)
print(f"Zero usage valid: input={zero_usage.input_tokens}, total={zero_usage.total_tokens}")

---
## 7. LLMResponse

The **normalized response** from any provider. This is the key abstraction — regardless of whether you're calling Anthropic, OpenAI, or Ollama, you get the same response shape.

| Field | Type | Purpose |
|-------|------|---------|
| `content` | `str \| None` | Text response (None during pure tool calls) |
| `tool_calls` | `list[ToolCall]` | Tools the LLM wants to execute |
| `usage` | `Usage` | Token counts |
| `model` | `str` | Which model responded |
| `stop_reason` | `str` | Why the LLM stopped (`"end_turn"`, `"tool_use"`, etc.) |
| `thinking` | `str \| None` | Extended thinking content (if supported) |
| `raw` | `Any` | Raw provider response (for debugging only) |

In [ ]:
# Scenario 1: Normal text response (end of conversation)
text_response = LLMResponse(
    content="The weather in Austin is 75F and sunny.",
    usage=Usage(input_tokens=500, output_tokens=20, total_tokens=520),
    model="claude-sonnet-4-20250514",
    stop_reason="end_turn",
)
print(f"Content: {text_response.content}")
print(f"Stop reason: {text_response.stop_reason}")
print(f"Tool calls: {text_response.tool_calls}  (empty = no tools needed)")
print(f"Model: {text_response.model}")

In [ ]:
# Scenario 2: Tool use response (agent loop continues)
tool_response = LLMResponse(
    content=None,  # No text when doing pure tool calls
    tool_calls=[
        ToolCall(id="c1", name="get_weather", arguments={"city": "Austin"}),
        ToolCall(id="c2", name="get_time", arguments={"timezone": "CST"}),
    ],
    usage=Usage(input_tokens=800, output_tokens=50, total_tokens=850),
    model="claude-sonnet-4-20250514",
    stop_reason="tool_use",
)
print(f"Content: {tool_response.content}  (None = LLM wants to use tools)")
print(f"Stop reason: {tool_response.stop_reason}")
print(f"Tool calls: {len(tool_response.tool_calls)}")
for tc in tool_response.tool_calls:
    print(f"  -> {tc.name}({tc.arguments})")

In [ ]:
# Scenario 3: Mixed response (text + tool calls)
# Some models return text alongside tool calls
mixed_response = LLMResponse(
    content="Let me look that up for you.",
    tool_calls=[
        ToolCall(id="c1", name="search", arguments={"q": "test"})
    ],
    usage=Usage(input_tokens=50, output_tokens=30, total_tokens=80),
    model="test-model",
    stop_reason="tool_use",
)
print(f"Has content AND tool calls:")
print(f"  Content: {mixed_response.content!r}")
print(f"  Tool calls: {len(mixed_response.tool_calls)}")

In [ ]:
# The agentic loop pattern: check stop_reason to decide what to do
def show_agent_decision(response: LLMResponse):
    """Simulate what an agent does with an LLMResponse."""
    if response.stop_reason == "end_turn":
        print(f"DONE -> Final answer: {response.content}")
    elif response.stop_reason == "tool_use":
        print(f"TOOL LOOP -> Execute {len(response.tool_calls)} tool(s), then call complete() again")
        for tc in response.tool_calls:
            print(f"  Execute: {tc.name}({tc.arguments})")
    else:
        print(f"UNKNOWN stop_reason: {response.stop_reason}")

print("--- Text response ---")
show_agent_decision(text_response)

print("\n--- Tool response ---")
show_agent_decision(tool_response)

---
## 8. LLMProvider (Abstract Base Class)

Every adapter (Anthropic, OpenAI, Ollama) must implement this interface.

It defines exactly two methods:
- `complete()` — send messages + tools, get back an `LLMResponse`
- `validate_config()` — check that the provider is properly configured

In [ ]:
# You CANNOT instantiate LLMProvider directly — it's abstract
try:
    provider = LLMProvider()
except TypeError as e:
    print(f"Cannot instantiate ABC: {e}")

In [ ]:
# But you CAN create a concrete subclass
class MockProvider(LLMProvider):
    name = "mock"

    async def complete(self, messages, tools=None, **kwargs):
        return LLMResponse(
            content="I'm a mock response!",
            usage=Usage(input_tokens=10, output_tokens=5, total_tokens=15),
            model="mock-v1",
            stop_reason="end_turn",
        )

    def validate_config(self):
        return True

mock = MockProvider()
print(f"Provider name: {mock.name}")
print(f"Config valid: {mock.validate_config()}")
print(f"Is LLMProvider? {isinstance(mock, LLMProvider)}")

In [ ]:
# Call the mock provider (async)
import asyncio

async def demo_complete():
    messages = [
        Message(role="user", content="Hello!")
    ]
    response = await mock.complete(messages)
    print(f"Response: {response.content}")
    print(f"Model: {response.model}")
    print(f"Usage: {response.usage.total_tokens} tokens")

await demo_complete()

---
## 9. Serialization

All types are Pydantic models, so they serialize cleanly to/from dicts and JSON.

In [ ]:
# Serialize to dict
msg = Message(
    role="assistant",
    content=[
        TextBlock(text="Here's a tool call"),
        ToolUseBlock(id="c1", name="calc", arguments={"expr": "2+2"}),
    ]
)

as_dict = msg.model_dump()
print(f"As dict:\n{json.dumps(as_dict, indent=2)}")

In [ ]:
# Serialize to JSON string
as_json = msg.model_dump_json(indent=2)
print(f"As JSON:\n{as_json}")

In [ ]:
# Deserialize from dict
reconstructed = Message.model_validate(as_dict)
print(f"Reconstructed: role={reconstructed.role}, blocks={len(reconstructed.content)}")
print(f"Same data? {reconstructed == msg}")

---
## 10. load_model() Placeholder

`load_model()` is the public API entry point. It's defined but not implemented yet (Step 6).

This is intentional — the type layer comes first, then config, then adapters, THEN the registry that ties it together.

In [ ]:
try:
    model = load_model("anthropic")
except NotImplementedError as e:
    print(f"Expected: {e}")
    print(f"\nThis will work after Step 6 (Provider Registry).")

---
## Summary

Step 1 built the **foundation contract**:

```
exceptions.py  ->  ArcLLMError, ArcLLMParseError, ArcLLMConfigError
types.py       ->  ContentBlock variants, Message, Tool, ToolCall, Usage, LLMResponse, LLMProvider
__init__.py    ->  Public API surface (all exports + load_model placeholder)
```

**Key design decisions:**
- Pydantic v2 for type safety and validation
- Discriminated union on `type` field for content blocks
- `str | list[ContentBlock]` for message content (flexibility)
- `dict[str, Any]` for tool parameters (raw JSON Schema, no over-abstraction)
- Arguments always parsed to dict (adapter's job, not the type's)
- ABC for providers (enforces interface contract)
- Fail-fast exceptions with raw data attached